In [ ]:
using LowLevelFEM
import LowLevelFEM as FEM
gmsh.initialize()

In [ ]:
l = 100
n = 100

In [ ]:
gmsh.parser.setNumber("length", [l])
gmsh.parser.setNumber("elements", [n])

In [ ]:
gmsh.merge("rod.geo")

In [ ]:
mat = FEM.material("body", E=2e5, ν=0)
problem = FEM.Problem([mat], type=:PlaneStress, bandwidth=:RCMK)

In [ ]:
supp1 = FEM.displacementConstraint("left", ux=0)
supp2 = FEM.displacementConstraint("bottom", uy=0)
supp3 = FEM.displacementConstraint("right", ux=0)

In [ ]:
#load = FEM.load("right", fx=1)
#u = FEM.solveDisplacement(problem, [load], [supp1, supp2])
#FEM.showDoFResults(problem, u, :ux, visible=true)

In [ ]:
K = FEM.stiffnessMatrix(problem)

In [ ]:
M = FEM.massMatrix(problem)

In [ ]:
Tmin = FEM.smallestPeriodTime(K, M)

In [ ]:
u0 = FEM.initialDisplacement(problem, "body", ux=0, uy=0);

In [ ]:
v0 = FEM.initialVelocity(problem, "body", vx=1000)
FEM.initialVelocity!(problem, "right", v0, vx=0)

In [ ]:
ωₘₐₓ = 2π / Tmin
C = FEM.dampingMatrix(K, M, ωₘₐₓ, ξ=0);

In [ ]:
dummyLoad = FEM.load("right", fx=0)
f = FEM.loadVector(problem, [dummyLoad]);

In [ ]:
FEM.applyBoundaryConditions!(problem, K, M, C, f, [supp2, supp3]);

In [ ]:
Δt = Tmin / π
c = √(problem.material[1].E / problem.material[1].ρ);
T = 2l / c;

In [ ]:
u, v, t = FEM.CDM(K, M, C, f, u0, v0, T, Δt * 1);

In [ ]:
S0 = FEM.solveStress(problem, u);

In [ ]:
S = FEM.solveStress(problem, u, DoFResults=true)

In [ ]:
S = FEM.elementsToNodes(problem, S0)

In [ ]:
Serr = FEM.fieldError(problem, S0)

In [ ]:
sx = FEM.showDoFResults(problem, S, :sx, t=t);

In [ ]:
serr = FEM.showDoFResults(problem, Serr, :sx, name="error", t=t);

In [ ]:
psx = FEM.plotOnPath(problem, "path", sx, points=n + 1, visible=true)
psx2, xy = FEM.plotOnPath(problem, "path", sx, points=(n + 1) * 10, step=length(t) ÷ 4, plot=true);

In [ ]:
using Plots
#plotlyjs()
plot(xy)

In [ ]:
c * Tmin

In [ ]:
length(t)

In [ ]:
Φ = FEM.solveModalAnalysis(problem, constraints=[supp1, supp2], n=301)
f = Φ.f

In [ ]:
fdofs = FEM.freeDoFs(problem, [supp1, supp2])

In [ ]:
cdofs = FEM.constrainedDoFs(problem, [supp1, supp2])

In [ ]:
dof = problem.non * problem.pdim

In [ ]:
dof - length(cdofs)

In [ ]:
fdofs = setdiff(1:dof, cdofs)

In [ ]:
Φ.ϕ' * Φ.ϕ

In [ ]:
q = Φ.ϕ' * u

In [ ]:
anim = @animate for i ∈ 1:200
    plot(Φ.f, abs.(q[:, i]))
end
mp4(anim, "anim_modal.mp4", fps=15)

In [ ]:
ff(x) = 2e3 / x
q1 = max.(min.(abs.(ff.(Φ.f .- 5e4)), 0.1), 0)
plot(Φ.f, q1)

In [ ]:
#ff(x) = 5e4 / x
#q1 = abs.(ff.(Φ.f))
u1 = Φ.ϕ * q1
S1 = FEM.solveStress(problem, u1)
S11 = FEM.elementsToNodes(problem, S1)
sx1 = FEM.showDoFResults(problem, S11, :sx)
psx3 = FEM.plotOnPath(problem, "path", sx1, points=100)
psx4, xy = FEM.plotOnPath(problem, "path", sx1, points=100, plot=true);
plot(xy)

In [ ]:
gmsh.fltk.openTreeItem("0Modules/Post-processing")

In [ ]:
gmsh.fltk.run()

In [ ]:
gmsh.finalize()